In [ ]:
%%sh
#!/usr/bin/env bash
docker system prune --force > /dev/null 2>&1

# The name of our algorithm
algorithm_name=wpi-test

cd testing

chmod -R +x coral/

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email) 2> /dev/null

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --no-cache # > /dev/null 2>&1
docker tag ${algorithm_name} ${fullname} > /dev/null 2>&1

docker push ${fullname} #> /dev/null 2>&1